In [12]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K

IMAGE_SHAPE = [94,24]
CHARS = "ABCDEFGHIJKLMNPQRSTUVWXYZ0123456789" # exclude I, O
CHARS_DICT = {char:i for i, char in enumerate(CHARS)}
DECODE_DICT = {i:char for i, char in enumerate(CHARS)}
NUM_CLASS = len(CHARS)+1



In [13]:
def CTCLoss(y_true, y_pred):
    # Compute the training-time loss value
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    
    return loss

In [14]:
class small_basic_block(keras.layers.Layer):

    def __init__(self,out_channels,name=None,**kwargs):
        super().__init__(**kwargs)
        out_div4=int(out_channels/4)
        self.main_layers = [
            keras.layers.Conv2D(out_div4,(1,1),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.Conv2D(out_div4,(3,1),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.Conv2D(out_div4,(1,3),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.Conv2D(out_channels,(1,1),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
        ]  
    
    def call(self,input):
        x = input
        for layer in self.main_layers:
            x = layer(x)
        return x

In [15]:
#test this later

class global_context(keras.layers.Layer):
    def __init__(self,kernel_size,stride,**kwargs):
        super().__init__(**kwargs)
        self.ksize = kernel_size
        self.stride = stride


    def call(self, input):
        x = input 
        avg_pool = keras.layers.AveragePooling2D(pool_size=self.ksize,strides=self.stride,padding='same')(x)
        sq = keras.layers.Lambda(lambda x: tf.math.square(x))(avg_pool)
        sqm = keras.layers.Lambda(lambda x: tf.math.reduce_mean(x))(sq)
        out = keras.layers.Lambda(lambda x: tf.math.divide(x[0], x[1]))([avg_pool , sqm])
        #out = keras.layers.Lambda(lambda x: K.l2_normalize(x,axis=1))(avg_pool)
        return out

In [16]:
class LPRnet(keras.Model):
    def __init__(self, input_shape=(24,94,3), **kwargs):
        super(LPRnet, self).__init__(**kwargs)
        self.input_layer = keras.layers.Input(input_shape)
        self.cnn_layers= [
            keras.layers.Conv2D(64,kernel_size = (3,3),strides=1,padding='same',name='main_conv1',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(name='BN1'),
            keras.layers.ReLU(name='RELU1'),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(1,1),name='maxpool2d_1',padding='same'),
            small_basic_block(128),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(1,2),name='maxpool2d_2',padding='same'),
            small_basic_block(256),
            small_basic_block(256),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(1,2),name='maxpool2d_3',padding='same'),
            keras.layers.Dropout(0.5),
            keras.layers.Conv2D(256,(4,1),strides=1,padding='same',name='main_conv2',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.Dropout(0.5),
            keras.layers.Conv2D(NUM_CLASS,(1,13),padding='same',name='main_conv3',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),  
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
        ]
        self.out_layers = [
            keras.layers.Conv2D(NUM_CLASS,kernel_size=(1,1),strides=(1,1),padding='same',name='conv_out',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            #keras.layers.BatchNormalization(),
            #keras.layers.ReLU(),
        ]
        #self.conv_out = keras.layers.Conv2D(NUM_CLASS,kernel_size=(1,1),strides=(1,1),padding='same',name='conv_out')
        self.out = self.call(self.input_layer)
        super(LPRnet, self).__init__(
            inputs=self.input_layer,
            outputs=self.out,
            **kwargs)

    def call(self,inputs,training=False):
        x = inputs
        layer_outputs = []
        for layer in self.cnn_layers:
            x = layer(x)
            layer_outputs.append(x)
        scale1 = global_context((1,4),(1,4))(layer_outputs[0])
        scale2 = global_context((1,4),(1,4))(layer_outputs[4])
        scale3 = global_context((1,2),(1,2))(layer_outputs[6])
        scale5 = global_context((1,2),(1,2))(layer_outputs[7])
        #scale4 = keras.layers.Lambda(lambda x: K.l2_normalize(x,axis=1))(x)
        sq = keras.layers.Lambda(lambda x: tf.math.square(x))(x)
        sqm = keras.layers.Lambda(lambda x: tf.math.reduce_mean(x))(sq)
        scale4 = keras.layers.Lambda(lambda x: tf.math.divide(x[0], x[1]))([x , sqm])
        gc_concat = keras.layers.Lambda(lambda x: tf.concat([x[0], x[1], x[2], x[3], x[4]],3))([scale1, scale2, scale3, scale5,scale4])
        for layer in self.out_layers:
            gc_concat = layer(gc_concat)
        logits = keras.layers.Lambda(lambda x: tf.math.reduce_mean(x[0],axis=1))([gc_concat])
        #transposed_logs = keras.layers.Lambda(lambda x: tf.transpose(x,(1,0,2)))(logits)
        logits = keras.layers.Softmax()(logits)
        return logits

In [17]:
model = LPRnet()

In [18]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=2000,
    decay_rate=0.9
)

In [19]:


model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),loss =CTCLoss)
for layer in model.layers:
    print(layer.name,layer.output_shape)

input_2 [(None, 24, 94, 3)]
main_conv1 (None, 24, 94, 64)
BN1 (None, 24, 94, 64)
RELU1 (None, 24, 94, 64)
maxpool2d_1 (None, 24, 94, 64)
small_basic_block_3 (None, 24, 94, 128)
maxpool2d_2 (None, 24, 47, 128)
small_basic_block_4 (None, 24, 47, 256)
small_basic_block_5 (None, 24, 47, 256)
maxpool2d_3 (None, 24, 24, 256)
dropout_2 (None, 24, 24, 256)
main_conv2 (None, 24, 24, 256)
batch_normalization_26 (None, 24, 24, 256)
re_lu_26 (None, 24, 24, 256)
dropout_3 (None, 24, 24, 256)
main_conv3 (None, 24, 24, 36)
batch_normalization_27 (None, 24, 24, 36)
re_lu_27 (None, 24, 24, 36)
lambda_5 (None, 24, 24, 36)
lambda_6 ()
global_context_4 (None, 24, 24, 64)
global_context_5 (None, 24, 24, 128)
global_context_6 (None, 24, 24, 256)
global_context_7 (None, 24, 24, 256)
lambda_7 (None, 24, 24, 36)
lambda_8 (None, 24, 24, 740)
conv_out (None, 24, 24, 36)
lambda_9 (None, 24, 36)
softmax_1 (None, 24, 36)


In [20]:
model.summary()

Model: "lp_rnet_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 24, 94, 3)]  0                                            
__________________________________________________________________________________________________
main_conv1 (Conv2D)             (None, 24, 94, 64)   1792        input_2[0][0]                    
__________________________________________________________________________________________________
BN1 (BatchNormalization)        (None, 24, 94, 64)   256         main_conv1[0][0]                 
__________________________________________________________________________________________________
RELU1 (ReLU)                    (None, 24, 94, 64)   0           BN1[0][0]                        
__________________________________________________________________________________________

In [27]:
import glob
import cv2
data = []
labels = []
val_data = []
val_labels = []

GEN_VAL_PATH = glob.glob('./valid/*.jpg')
LP_VAL_PATH = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\TFODCourse\\test frames\\filtered_plates\\driving_quezoncity.mp4\\*.png')

#for file in glob.glob('./test/*.jpg'):
#    label=file.split('\\')[1]
#    image = cv2.imread(file,cv2.IMREAD_COLOR)
#    image = cv2.resize(image,(94,24))
#    data.append(image/256)
#    labels.append([CHARS_DICT[i] for i in label.split('_')[0]])
for file in LP_VAL_PATH:
    label=file.split('\\')[-1]
    image = cv2.imread(file,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(94,24))
    val_data.append(image/256)
    val_labels.append([CHARS_DICT[i] for i in label.split('_')[0]])


In [28]:
#training_set = np.array(data,dtype=np.float32)
#training_labels = np.array(labels)
#ragged = tf.ragged.constant(training_labels).to_tensor()
#dataset = tf.data.Dataset.from_tensor_slices((training_set,ragged)).shuffle(640).repeat(100).batch(64)


val_training_set = np.array(val_data,dtype=np.float32)
val_training_labels = np.array(val_labels)
val_ragged = tf.ragged.constant(val_training_labels).to_tensor()
val_dataset = tf.data.Dataset.from_tensor_slices((val_training_set,val_ragged)).shuffle(640).repeat(100).batch(64)

C:\Users\carlos\AppData\Local\Temp/ipykernel_17732/1567297501.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  val_training_labels = np.array(val_labels)


In [29]:
from gen_plates_keras import *
gen = ImageGenerator()
def generate_dataset(num = 100):
    data, labels = gen.generate_images(num)
    gen_labels = []
    for label in labels:
        gen_labels.append([CHARS_DICT[i] for i in label.split('_')[0]])
    pics =np.array(data)
    labels = np.array(labels)
    training_set = np.array(pics,dtype=np.float32)
    training_labels = np.array(gen_labels)
    ragged = tf.ragged.constant(training_labels).to_tensor()
    dataset = tf.data.Dataset.from_tensor_slices((training_set,ragged)).shuffle(640).repeat(100).batch(64)
    return dataset
standalone_dataset = generate_dataset(num=100)

C:\Users\carlos\AppData\Local\Temp/ipykernel_17732/2817659380.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training_labels = np.array(gen_labels)


In [30]:
import wandb
from wandb.keras import WandbCallback
wandb.init(project="my-test-project", entity="clsandoval")
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 400,
  "batch_size": 64
}

def generate_val_set(num=400):
        val_data_gen, val_labels_gen = gen.generate_images(num)
        val_labels_encoded = []
        val_data_encoded = []
        for image,label in zip(val_data_gen, val_labels_gen):
                val_data_encoded.append(np.expand_dims(image,axis=0))
                val_labels_encoded.append([CHARS_DICT[i] for i in label.split('_')[0]])
        return val_data_encoded,val_labels_encoded
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',patience=10)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: clsandoval (use `wandb login --relogin` to force relogin)


In [31]:
#standalone_dataset = generate_dataset(num=10000)
#val_dataset = generate_dataset(1000)
for j in range(100):
    dataset = generate_dataset(num=32000)
    model.fit(dataset,validation_data=val_dataset,validation_steps=5,epochs=10,steps_per_epoch=50,callbacks=[WandbCallback()])
    print(j,flush=True)

C:\Users\carlos\AppData\Local\Temp/ipykernel_17732/2817659380.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training_labels = np.array(gen_labels)


Epoch 1/10
50/50 [==============================] - 19s 171ms/step - loss: 33.5407 - val_loss: 100.2391


wandb: ERROR Can't save model, h5py returned error: Layer small_basic_block has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/10
50/50 [==============================] - 7s 131ms/step - loss: 22.0774 - val_loss: 100.2402
Epoch 3/10
50/50 [==============================] - 7s 131ms/step - loss: 21.1221 - val_loss: 100.2412
Epoch 4/10
50/50 [==============================] - 8s 153ms/step - loss: 20.6700 - val_loss: 100.2361
Epoch 5/10
50/50 [==============================] - 8s 165ms/step - loss: 20.3662 - val_loss: 98.0105
Epoch 6/10
50/50 [==============================] - 7s 131ms/step - loss: 20.2234 - val_loss: 93.2701
Epoch 7/10
50/50 [==============================] - 7s 132ms/step - loss: 19.6608 - val_loss: 92.3614
Epoch 8/10
50/50 [==============================] - 6s 126ms/step - loss: 18.7067 - val_loss: 87.8867
Epoch 9/10
50/50 [==============================] - 6s 126ms/step - loss: 17.3349 - val_loss: 82.6365
Epoch 10/10
50/50 [==============================] - 7s 131ms/step - loss: 14.4598 - val_loss: 72.9066
0


C:\Users\carlos\AppData\Local\Temp/ipykernel_17732/2817659380.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training_labels = np.array(gen_labels)


...

Epoch 8/10
50/50 [==============================] - 6s 126ms/step - loss: 0.7540 - val_loss: 27.8242
Epoch 9/10
50/50 [==============================] - 6s 126ms/step - loss: 0.7277 - val_loss: 26.0752
Epoch 10/10
50/50 [==============================] - 6s 126ms/step - loss: 0.7311 - val_loss: 22.0301
9
Epoch 1/10
50/50 [==============================] - 7s 126ms/step - loss: 0.7357 - val_loss: 27.4729
Epoch 2/10
50/50 [==============================] - 6s 125ms/step - loss: 0.7241 - val_loss: 27.7528
Epoch 3/10
50/50 [==============================] - 6s 126ms/step - loss: 0.6917 - val_loss: 23.8029
Epoch 4/10
50/50 [==============================] - 6s 125ms/step - loss: 0.7403 - val_loss: 23.5269
Epoch 5/10
50/50 [==============================] - 6s 126ms/step - loss: 0.7149 - val_loss: 32.4395
Epoch 6/10
50/50 [==============================] - 6s 125ms/step - loss: 0.7242 - val_loss: 25.5413
Epoch 7/10
50/50 [==============================] - 6s 126ms/step - loss: 0.7170 - val_l

wandb: ERROR Can't save model, h5py returned error: Layer small_basic_block has arguments in `__init__` and therefore must override `get_config`.


Epoch 10/10
50/50 [==============================] - 6s 125ms/step - loss: 0.6472 - val_loss: 27.2942
13
Epoch 1/10
50/50 [==============================] - 6s 124ms/step - loss: 0.7195 - val_loss: 26.2126
Epoch 2/10
50/50 [==============================] - 6s 124ms/step - loss: 0.6488 - val_loss: 25.3450
Epoch 3/10
50/50 [==============================] - 6s 125ms/step - loss: 0.7081 - val_loss: 23.3087
Epoch 4/10
50/50 [==============================] - 6s 124ms/step - loss: 0.6517 - val_loss: 25.8681
Epoch 5/10
50/50 [==============================] - 6s 124ms/step - loss: 0.6432 - val_loss: 23.4375
Epoch 6/10
50/50 [==============================] - 6s 125ms/step - loss: 0.6056 - val_loss: 22.8156
Epoch 7/10
50/50 [==============================] - 6s 124ms/step - loss: 0.6550 - val_loss: 22.3424
Epoch 8/10
50/50 [==============================] - 6s 124ms/step - loss: 0.6368 - val_loss: 22.8148
Epoch 9/10
50/50 [==============================] - 6s 124ms/step - loss: 0.6119 - val_

wandb: ERROR Can't save model, h5py returned error: Layer small_basic_block has arguments in `__init__` and therefore must override `get_config`.


Epoch 5/10
50/50 [==============================] - 6s 124ms/step - loss: 0.6251 - val_loss: 23.5535
Epoch 6/10
50/50 [==============================] - 6s 124ms/step - loss: 0.6422 - val_loss: 23.3743
Epoch 7/10
50/50 [==============================] - 6s 124ms/step - loss: 0.6091 - val_loss: 25.1652
Epoch 8/10
50/50 [==============================] - 6s 124ms/step - loss: 0.6248 - val_loss: 23.9818
Epoch 9/10
50/50 [==============================] - 6s 124ms/step - loss: 0.6028 - val_loss: 25.0917
Epoch 10/10
50/50 [==============================] - 6s 124ms/step - loss: 0.6763 - val_loss: 26.3836
15


In [13]:
import glob
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\TFODCourse\\test frames\\filtered_plates\\driving_quezoncity.mp4\\*.png')
for file in real_images:
    image = cv2.imread(file)
    test_image = cv2.resize(image,(94,24))/256
    test_image = np.expand_dims(test_image,axis=0)
    preds = model.predict(test_image) 
    decoded = tf.keras.backend.ctc_decode(preds,(24,))
    for i in np.array(decoded[0]).reshape(24):
        if i >-1:
            print(DECODE_DICT[i],end='')
    print(" "+ file.split('\\')[-1].split('_')[0])

367NPGAA 367NPG
A0AJ707 A0A7707
AAAZ8GH AAA2B0M
AAB3L32 AAB3432
AIBTL32 AAB3432
AAH773T AAH7731
RH01535A AAHA812
AA33T5TA AAJ3751
AA37B0S AAJ7809
AAKGB5G AAK6856
AB82G45AA ABB2645
GB8488GA ABB4860
LAB03BS3 ABD3833
ABEG72A ABE8672
ABG2SGG ABG2566
ABH7D7ZA ABH7077
ABR5SMMAA ABR5577
ABVSG3D ABVS850
ADAJ082AA ADA1082
ADA7082A ADA1082
ADHJ88A ADH1388
UAKA3B55 AKA3855
IKA43GSA AKA4369
AKAGGS2 AKA6692
613Z74LA APA3474
ARAS0BS ARA9085
AS14GGS7 ASA6697.png
ASAGGS7 ASA6697
AH050SAA AWD305
FBCLS413S BCL5413
BEFBT2 BEF812
CA86472A CAB6472
CAN164ZA CAN1641
C01ZZ8A CAU1176
CAVSZB6 CAV5486
CAVB710 CAV8710
DA5Z54B DAG3548
DBZ9983S DBZ9983
F2XS2ZA F2X322
B6SBJA G1C693
HNEXSSZA HBE382
IAB649SA IAB6499
NACSBS6 NAC5836
NAJ1S779A NAJ3779
CAJJZ79 NAJ3779
NAKS39SA NAK5393
NANJZ95L NAN1795
NATSBZB NAT5828
NBD9D1ZA NBD9011
NBGS026A NBG5026
TNB03JBQ4 NBO3804
NB0S4A NBO3814
CNB0ZZ81 NBQ3281
NBZBG81 NBZ8081
NCL6S22 NCL6522
ND02ZZ41 NDD2441
NDE636 NDE4636
ND0G0S NDG6005
NDH5506 NDH5506
NK1722LA NDK1722
NDPJ545AA N

In [15]:
np.array(labels).shape

C:\Users\carlos\AppData\Local\Temp/ipykernel_15568/4033116927.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.array(labels).shape


(8200,)

In [13]:
for layer in small_basic_block(256).layers:
    print(layer)

AttributeError: 'small_basic_block' object has no attribute 'layers'

In [17]:
for layer in small_basic_block(256).main_layers:
    print(layer)

In [14]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('keras_lprnet1.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\carlos\AppData\Local\Temp\tmp71ax2byh\assets


INFO:tensorflow:Assets written to: C:\Users\carlos\AppData\Local\Temp\tmp71ax2byh\assets


In [2]:
interpreter = tf.lite.Interpreter('keras_lprnet2.tflite')

In [9]:
import numpy as np
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [23]:
import glob,cv2 
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\TFODCourse\\test frames\\driving_quezoncity.mp4\\*.png')
for file in real_images:
    image = cv2.imread(file)
    test_image = cv2.resize(image,(94,24))/256
    test_image = np.expand_dims(test_image,axis=0)
    test_image = test_image.astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], test_image)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    decoded = tf.keras.backend.ctc_decode(output_data,(24,))
    for i in np.array(decoded[0]).reshape(24):
        if i >-1:
            print(DECODE_DICT[i],end='')
    print(" "+ file.split('\\')[-1].split('_')[0])

UU01JIBWAA 00B
D0CX5IBAA 00C6518
D0CX5JIGAA 00C65TZB
U0LX5IGA 00CA5TB
UUCX5ZBAA 00CS5T8
U0LX5IGAA 00CS5TB
U0LX5IB 00CS5TB
U0I3IBAA 00ETB
UUVLIAAA 00FE
H304ZLAAA 00L
37SZAA 00TEZJ
US1Z7IAA 011ZE4
U0IIZZAA 01P
FB29AA 0259
F082AAAA 02T
VU8IZBA 067B
U081ZBA 067B
UU8ZBAA 067B
M0A3ZBA 06B
UU4BZIBAA 06B
UU0IBZBA 06B
UVU8DZBA 06B
UUU45IBAA 06B
U80YBIB 06TB
UU0ABZBA 06TB
UVUKDZBA 06TB
UV083ZBA 06TPB
U0C65TG 08C5T8
U0C65ZGA 08C5T8
U0CX5IGA 08CA5TB
U0IBAA 0AB
U0CX5JBA 0BE5T8
K5AAA 0EE
W40IABZB 0GB
LV6N23VUVNA 0GHSUHUQ
HEGTQAA 0HFEETS
NHM8IZB 0MBAEZE
5IHLA 0MF
UN2IDAB 0RTCVTA
HWUE57CA 0UL54
TY23SAA 0Y255
VY25SAA 0Y2J5
L1176AAA 1176T
FZ17BAA 1176
UU05DAIA 11LEJ
ZAA 122ZT
ZAAA 122ZT
ZZ5AA 12Z5
1365LA 1365T
V16AA 16
C16AAAA 16
Z795ILAA 1795T
LE84BZAA 1ALTUM
03GZA 1EEZ
UEIIAA 1EFJ
HWENJBZA 1ELEZ
U662TUTAA 1ET0T
FZE4BAAA 1FGE
FN44ZZ 1MFN44Z4
BD4ZAAA 1RZ8J
5AAA 1T35Z
U633UAA 1TST0T
FZELAAA 1YEE
I722IAAA 1Z22TZ
ZLAAA 1Z
DQJ1ZSAA 212Z2GT
UU4Z5EAA 21AA2RBD
UWZ1AA 21R2EB
Z7AA 2291TZ
B74AA 229ZT
ZAA 229Z
ZAA